In [ ]:
# Xena prep
    # for synthesis app example files preparation
    # https://github.com/ohsu-comp-bio/synthesis
    
# 1. Select features with mean absolute deviation across all samples
    # reduce from ~60k to 5k features
    
# 2. Build X_train
    # Remove samples designated for synthesis
    
# 3. Build X_synth, a PAM50-labeled synthesis file
    # also used for model conditioning (fine-tuning)

# 4. Normalize within each feature, seperately within Xtrain and X_synth
    # based on method of norming within each cohort

In [1]:
import pandas as pd
import time

### GDC-PANCAN.htseq_fpkm-uq.tsv.gz
#### and
### brca-pam50
#### are the files downloaded from xena
https://xenabrowser.net/datapages/?dataset=GDC-PANCAN.htseq_fpkm-uq.tsv&host=https%3A%2F%2Fgdc.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443  
https://xenabrowser.net/datapages/?dataset=brca%2Fpam50&host=https%3A%2F%2Fatacseq.x[…].net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443 

In [2]:
samples_with_p50_labels = pd.read_csv(
    'i_o/xena/brca-pam50',
    sep = '\t', index_col = 0)

In [3]:
samples_with_p50_labels.PAM50.unique()

array(['LumB', 'Basal', 'LumA', 'Her2', 'Normal', 'Unknown'], dtype=object)

In [4]:
samples_with_p50_labels.PAM50.value_counts()

LumB       47
LumA       34
Basal      30
Her2       22
Unknown     7
Normal      1
Name: PAM50, dtype: int64

In [5]:
label_keep = ['LumB', 'Basal', 'LumA', 'Her2']

In [6]:
p50_label_cut = samples_with_p50_labels[samples_with_p50_labels.PAM50.isin(label_keep)]

In [7]:
len(p50_label_cut)

133

In [8]:
p50_label_unq = p50_label_cut[~p50_label_cut.index.duplicated(keep='first')]

In [16]:
p50_label_unq.head(1)

,PAM50
sample,
TCGA-A7-A13F-01A,LumB


In [9]:
p50_label_unq.PAM50.value_counts() # 11 Her2 samples, rarest subtype

LumB     25
LumA     18
Basal    15
Her2     11
Name: PAM50, dtype: int64

In [10]:
syn_lst = list(p50_label_unq.index) # Samples designated for synthesis

In [11]:
len(syn_lst) # check, use for spliting gexp main_frame into train_frame and synth_frame

69

In [12]:
file_read_start = time.time()
X_in = pd.read_csv('i_o/xena/GDC-PANCAN.htseq_fpkm-uq.tsv.gz', sep = '\t', index_col = 0)
print(time.time() - file_read_start)

124.09854388237


In [13]:
main_frame = X_in.T

In [14]:
main_frame.shape # (11768, 60483)

(11768, 60483)

In [15]:
main_frame.head(1)

xena_sample,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
TCGA-OR-A5JP-01A,0.0,10.689655,18.536987,0.0,17.847476,17.092449,0.0,7.645763,21.628175,10.917522,...,0.0,0.0,0.0,12.722942,11.773066,19.730227,11.10208,0.0,11.102346,0.0


In [ ]:
# Check if all these samples have a ~60k feature expression vector from the X_main gexp file
main_frame.loc[syn_lst, :] # Two PAM50-labeled samples with no expression vector

In [26]:
# Remove two PAM50 samples that don't map to the expression file from the synth list
# then extract the synth_X from the main frame

In [17]:
syn_lst.remove('TCGA-AQ-A04L-01A')
syn_lst.remove('TCGA-BH-A0C1-01A')

In [ ]:
# Extract the features for the unique labled samples that map to the expression object

In [19]:
X_synth = main_frame.loc[syn_lst]

In [20]:
X_synth.head(1)

xena_sample,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
TCGA-A7-A13F-01A,0.0,6.397499,15.766939,0.0,16.468291,17.419663,0.0,0.0,22.030428,11.433228,...,0.0,7.094493,0.0,0.0,9.968097,18.488007,13.009847,0.0,10.88142,0.0


In [22]:
p50_label_gexp_map = p50_label_unq.loc[syn_lst]

In [25]:
p50_label_gexp_map.head(1)

,PAM50
sample,
TCGA-A7-A13F-01A,LumB


In [24]:
synth_labeled = pd.concat([p50_label_gexp_map, X_synth], axis = 1)

In [26]:
synth_labeled.head(1)

,PAM50,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
TCGA-A7-A13F-01A,LumB,0.0,6.397499,15.766939,0.0,16.468291,17.419663,0.0,0.0,22.030428,...,0.0,7.094493,0.0,0.0,9.968097,18.488007,13.009847,0.0,10.88142,0.0


In [27]:
synth_labeled.rename(columns={'PAM50': "Labels"}, inplace = True)

In [28]:
synth_labeled.to_csv('i_o/xena/xena_brca_n67.tsv', sep = '\t')

In [29]:
syn_chk = pd.read_csv('i_o/xena/xena_brca_n67.tsv', sep = '\t', index_col = 0)

In [30]:
syn_chk

,Labels,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
TCGA-A7-A13F-01A,LumB,0.000000,6.397499,15.766939,0.0,16.468291,17.419663,0.0,0.000000,22.030428,...,0.0,7.094493,0.0,0.000000,9.968097,18.488007,13.009847,0.0,10.881420,0.000000
TCGA-A7-A13E-01A,Basal,0.000000,9.240214,13.910406,0.0,14.706670,17.694489,0.0,10.496368,21.288673,...,0.0,8.721828,0.0,10.815075,11.090464,17.691870,13.100024,0.0,14.094683,0.000000
TCGA-BH-A0DP-01A,LumA,9.283496,0.000000,15.967342,0.0,15.662792,18.170825,0.0,11.242179,22.096182,...,0.0,0.000000,0.0,9.969659,10.019535,17.916953,12.019688,0.0,13.517229,0.000000
TCGA-A2-A0EW-01A,LumA,9.972301,8.447524,15.699358,0.0,16.097541,17.223890,0.0,7.750358,21.467985,...,0.0,8.414270,0.0,11.658351,9.709554,17.967774,12.014229,0.0,12.622058,8.069625
TCGA-A7-A0CH-01A,LumA,0.000000,7.469552,16.073231,0.0,16.189393,17.599560,0.0,8.502595,21.230527,...,0.0,0.000000,0.0,12.583035,11.633179,17.629331,11.868772,0.0,12.331275,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-C8-A12V-01A,Basal,0.000000,0.000000,16.243491,0.0,16.667266,18.219955,0.0,10.086592,21.094950,...,0.0,0.000000,0.0,11.884431,9.935449,18.662539,11.307709,0.0,15.175939,0.000000
TCGA-A2-A0YH-01A,LumB,9.011810,0.000000,16.047603,0.0,16.278690,17.637075,0.0,10.105028,22.074732,...,0.0,7.869402,0.0,10.696922,0.000000,18.708863,11.360699,0.0,13.414996,0.000000
TCGA-AO-A0JB-01A,Basal,0.000000,0.000000,17.654755,0.0,14.977014,17.464474,0.0,9.627344,22.594490,...,0.0,10.877640,0.0,14.031720,14.081643,18.270272,12.258621,0.0,14.612287,0.000000
TCGA-A2-A0T5-01A,LumA,0.000000,6.258729,15.247359,0.0,15.399994,18.233816,0.0,12.299640,22.500334,...,0.0,10.114606,0.0,9.777856,0.000000,18.081199,10.910544,0.0,13.614720,0.000000


### Synthesis file prepped for feature selection and normalization

### Build train file, run MAD 5k, normalize

In [39]:
X_in.shape # count all samples

(60483, 11768)

In [40]:
train_frame = main_frame.loc[~main_frame.index.isin(syn_lst)]
train_frame.shape

(11701, 60483)

In [41]:
11701 + 67 # all samples accounted for

11768

In [35]:
train_frame.head(1)

xena_sample,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
TCGA-OR-A5JP-01A,0.0,10.689655,18.536987,0.0,17.847476,17.092449,0.0,7.645763,21.628175,10.917522,...,0.0,0.0,0.0,12.722942,11.773066,19.730227,11.10208,0.0,11.102346,0.0


In [ ]:
del(X_in)

In [ ]:
del(main_frame)

In [51]:
write_time = time.time()
train_frame.to_csv('i_o/xena/xena_gexp_n11702.tsv', sep = '\t')
print(time.time() - write_time)

849.271585226059


In [ ]:
read_time = time.time()
trn_chk = pd.read_csv('i_o/xena/xena_gexp_n11702.tsv', sep = '\t', index_col = 0)
print(time.time() - read_time)

In [43]:
train_frame.insert(0, 'Labels', 'x_n')

In [44]:
train_frame.head(1)

xena_sample,Labels,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
TCGA-OR-A5JP-01A,x_n,0.0,10.689655,18.536987,0.0,17.847476,17.092449,0.0,7.645763,21.628175,...,0.0,0.0,0.0,12.722942,11.773066,19.730227,11.10208,0.0,11.102346,0.0


In [45]:
synth_labeled.head(1)

,Labels,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
TCGA-A7-A13F-01A,LumB,0.0,6.397499,15.766939,0.0,16.468291,17.419663,0.0,0.0,22.030428,...,0.0,7.094493,0.0,0.0,9.968097,18.488007,13.009847,0.0,10.88142,0.0


In [46]:
stack = pd.concat([train_frame, synth_labeled], axis = 0)

In [47]:
stack.shape

(11768, 60484)

In [48]:
stack.head(1)

,Labels,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
TCGA-OR-A5JP-01A,x_n,0.0,10.689655,18.536987,0.0,17.847476,17.092449,0.0,7.645763,21.628175,...,0.0,0.0,0.0,12.722942,11.773066,19.730227,11.10208,0.0,11.102346,0.0


In [49]:
stack.tail(1)

,Labels,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
TCGA-A2-A0YG-01A,LumB,0.0,8.411591,15.184759,0.0,16.249897,18.530273,0.0,12.972487,20.788958,...,0.0,8.792292,0.0,12.207135,0.0,18.839781,11.286543,0.0,12.045564,0.0


In [50]:
k = 5 # Multiple for n thousand features to select
mad_start = time.time()
features = (stack.iloc[:, 1:] - stack.iloc[:, 1:].mean()).abs().mean().sort_values(ascending=False)[:k*1000].index
print(time.time() - mad_start)

4.817362070083618


In [51]:
len(features) # check

5000

In [52]:
stack.Labels

TCGA-OR-A5JP-01A      x_n
TCGA-OR-A5JE-01A      x_n
TCGA-OR-A5JG-01A      x_n
TCGA-OR-A5L9-01A      x_n
TCGA-OR-A5JR-01A      x_n
                    ...  
TCGA-C8-A12V-01A    Basal
TCGA-A2-A0YH-01A     LumB
TCGA-AO-A0JB-01A    Basal
TCGA-A2-A0T5-01A     LumA
TCGA-A2-A0YG-01A     LumB
Name: Labels, Length: 11768, dtype: object

In [54]:
stack.index.unique() # no duplicates

Index(['TCGA-OR-A5JP-01A', 'TCGA-OR-A5JE-01A', 'TCGA-OR-A5JG-01A',
       'TCGA-OR-A5L9-01A', 'TCGA-OR-A5JR-01A', 'TCGA-OR-A5KU-01A',
       'TCGA-OR-A5LS-01A', 'TCGA-OR-A5J7-01A', 'TCGA-OR-A5JQ-01A',
       'TCGA-OR-A5JS-01A',
       ...
       'TCGA-A2-A0YL-01A', 'TCGA-C8-A12U-01A', 'TCGA-AO-A124-01A',
       'TCGA-BH-A1EV-01A', 'TCGA-A2-A0ES-01A', 'TCGA-C8-A12V-01A',
       'TCGA-A2-A0YH-01A', 'TCGA-AO-A0JB-01A', 'TCGA-A2-A0T5-01A',
       'TCGA-A2-A0YG-01A'],
      dtype='object', length=11768)

In [55]:
stack_5k = pd.concat(
    [stack.Labels, stack.loc[:, features]],
    axis = 1)

In [56]:
stack_5k.shape

(11768, 5001)

In [ ]:
# Feature-wise normalization done independently within the train file and the synthesis file
# Common mad features between train and synth for transfer learning

In [57]:
train_5k = stack_5k.loc[~stack_5k.index.isin(syn_lst)]

In [58]:
train_5k.shape

(11701, 5001)

In [59]:
synth_5k = stack_5k.loc[stack_5k.index.isin(syn_lst)]

In [60]:
synth_5k.shape

(67, 5001)

### normalize train

In [61]:
train_max_col_vals = train_5k.iloc[:, 1:].max()

In [62]:
X_train = (train_5k.iloc[:, 1:] / train_max_col_vals)

In [74]:
X_train.head(1)

,ENSG00000226278.1,ENSG00000274735.1,ENSG00000169474.4,ENSG00000211593.2,ENSG00000204894.4,ENSG00000169469.8,ENSG00000211905.1,ENSG00000237111.1,ENSG00000241781.2,ENSG00000239776.2,...,ENSG00000234184.4,ENSG00000253334.1,ENSG00000241105.1,ENSG00000257715.1,ENSG00000266206.1,ENSG00000267644.1,ENSG00000228525.1,ENSG00000203593.3,ENSG00000272218.1,ENSG00000250366.2
TCGA-OR-A5JP-01A,0.0,0.453327,0.709609,0.0,0.0,0.478471,0.0,0.0,0.485832,0.0,...,0.0,0.539001,0.0,0.0,0.0,0.658941,0.0,0.527925,0.0,0.0


In [64]:
X_train.shape

(11701, 5000)

In [65]:
train_5k.Labels

TCGA-OR-A5JP-01A        x_n
TCGA-OR-A5JE-01A        x_n
TCGA-OR-A5JG-01A        x_n
TCGA-OR-A5L9-01A        x_n
TCGA-OR-A5JR-01A        x_n
                       ... 
TARGET-50-PAJNZU-01A    x_n
TARGET-50-PAJNNR-01A    x_n
TARGET-50-PAJNTJ-02A    x_n
TARGET-50-PAECJB-01A    x_n
TARGET-50-PALFRD-01A    x_n
Name: Labels, Length: 11701, dtype: object

In [66]:
X_train_labeled = pd.concat([train_5k.Labels, X_train], axis = 1)

In [67]:
X_train_labeled.shape

(11701, 5001)

In [68]:
X_train_labeled.to_csv('i_o/xena/X_train_xena_2022-02-02.tsv', sep = '\t')

In [69]:
X_train_read_test = pd.read_csv('i_o/xena/X_train_xena_2022-02-02.tsv', sep = '\t', index_col = 0)

In [75]:
X_train_read_test.head(1)

,Labels,ENSG00000226278.1,ENSG00000274735.1,ENSG00000169474.4,ENSG00000211593.2,ENSG00000204894.4,ENSG00000169469.8,ENSG00000211905.1,ENSG00000237111.1,ENSG00000241781.2,...,ENSG00000234184.4,ENSG00000253334.1,ENSG00000241105.1,ENSG00000257715.1,ENSG00000266206.1,ENSG00000267644.1,ENSG00000228525.1,ENSG00000203593.3,ENSG00000272218.1,ENSG00000250366.2
TCGA-OR-A5JP-01A,x_n,0.0,0.453327,0.709609,0.0,0.0,0.478471,0.0,0.0,0.485832,...,0.0,0.539001,0.0,0.0,0.0,0.658941,0.0,0.527925,0.0,0.0


In [71]:
X_train_read_test.index.unique()

Index([    'TCGA-OR-A5JP-01A',     'TCGA-OR-A5JE-01A',     'TCGA-OR-A5JG-01A',
           'TCGA-OR-A5L9-01A',     'TCGA-OR-A5JR-01A',     'TCGA-OR-A5KU-01A',
           'TCGA-OR-A5LS-01A',     'TCGA-OR-A5J7-01A',     'TCGA-OR-A5JQ-01A',
           'TCGA-OR-A5JS-01A',
       ...
       'TARGET-50-PAJMKJ-01A', 'TARGET-50-CAAAAQ-11A', 'TARGET-50-PAKSCC-01A',
       'TARGET-50-PAJNSL-11A', 'TARGET-50-PAJPAU-01A', 'TARGET-50-PAJNZU-01A',
       'TARGET-50-PAJNNR-01A', 'TARGET-50-PAJNTJ-02A', 'TARGET-50-PAECJB-01A',
       'TARGET-50-PALFRD-01A'],
      dtype='object', length=11701)

In [ ]:
# End normalization of train file

### Normalize the tune file, put Labels back on from the drop 2 synth objectm

In [77]:
synth_max_col_vals = synth_5k.iloc[:, 1:].max()

In [78]:
X_synth = (synth_5k.iloc[:, 1:] / synth_max_col_vals)

In [80]:
X_synth_labeled = pd.concat([synth_5k.Labels, X_synth], axis = 1)

In [82]:
X_synth.to_csv('i_o/xena/X_synth_xena_2022-02-02.tsv', sep = '\t')

In [83]:
X_synth_read_test = pd.read_csv('i_o/xena/X_synth_xena_2022-02-02.tsv', sep = '\t', index_col = 0)

In [84]:
X_synth_read_test.head(1)

,ENSG00000226278.1,ENSG00000274735.1,ENSG00000169474.4,ENSG00000211593.2,ENSG00000204894.4,ENSG00000169469.8,ENSG00000211905.1,ENSG00000237111.1,ENSG00000241781.2,ENSG00000239776.2,...,ENSG00000234184.4,ENSG00000253334.1,ENSG00000241105.1,ENSG00000257715.1,ENSG00000266206.1,ENSG00000267644.1,ENSG00000228525.1,ENSG00000203593.3,ENSG00000272218.1,ENSG00000250366.2
TCGA-A7-A13F-01A,0.0,0.0,0.0,0.0,0.657671,0.414202,0.0,0.738717,0.0,0.746165,...,0.0,0.0,0.0,0.851262,0.0,0.653127,0.856256,0.748212,0.0,0.0
